In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5640576777068493205
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10761858375567468233
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
import pandas as pd


train = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/train.tsv', sep = '\t')
test = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/test.tsv', sep = '\t')
dev = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/dev.tsv', sep = '\t')

In [4]:
print(len(test))
print(len(dev))
len(train)


3000
2995


13998

In [5]:
# df = df[df['Y'].str.split().str.len().lt(4)]
train = train[train['tweet'].str.split().str.len().gt(10)]
test = test[test['tweet'].str.split().str.len().gt(10)]
dev = dev[dev['tweet'].str.split().str.len().gt(10)]
print(len(train))
print(len(test))
print(len(dev))

9447
2054
2026


In [6]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
train.tail()

,Unnamed: 0,tweet,label
13990,13992,@Crushscreenpart @alisonessex1 @AndrewRosindel...,1
13994,13996,need Respect love. China look love Respect. Af...,1
13995,13997,Centre made “grave mistake” announcing #lockdo...,1
13996,13998,Chinese experts noted harshness lockdown indic...,1
13997,13999,equipped take COVID-19 patients. supposed take...,1


In [8]:
from bs4 import BeautifulSoup
import re,string,unicodedata
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub(string.punctuation, '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(str(text))
    text = remove_between_square_brackets(str(text))
    text = remove_stopwords(str(text))
    return text


# s = train['tweet'][56]
# print(s)
# print(denoise_text(s.translate(str.maketrans('', '', string.punctuation))))
# # s = " ".join([word for word in denoise_text(s).lower().split() if word not in stop])
# # print(s)

In [9]:
import string

# s=str(s)
# s = s.translate(str.maketrans('', '', string.punctuation))
# s

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

#test["tweet"] = test["tweet"].str.lower().str.split()

train['tweet'] = train['tweet'].apply(lambda x: denoise_text(x.translate(str.maketrans('', '', string.punctuation))))
test['tweet'] = test['tweet'].apply(lambda x: denoise_text(x.translate(str.maketrans('', '', string.punctuation))))
dev['tweet'] = dev['tweet'].apply(lambda x: denoise_text(x.translate(str.maketrans('', '', string.punctuation))))

In [11]:
train.tail()

,Unnamed: 0,tweet,label
13990,13992,Crushscreenpart alisonessex1 AndrewRosindell T...,1
13994,13996,need Respect love China look love Respect Afri...,1
13995,13997,Centre made “grave mistake” announcing lockdow...,1
13996,13998,Chinese experts noted harshness lockdown indic...,1
13997,13999,equipped take COVID19 patients supposed take p...,1


In [12]:
len(train)

9447

In [13]:
from collections import Counter

top_100_list_train = list()
top_100_list_train = [item for item,_ in Counter(" ".join(train['tweet']).split()).most_common(100)]

top_100_list_test = list()
top_100_list_train = [item for item,_ in Counter(" ".join(test['tweet']).split()).most_common(100)]

top_100_list_dev = list()
top_100_list_train = [item for item,_ in Counter(" ".join(dev['tweet']).split()).most_common(100)]

train['2tweet'] = train['tweet'].apply(lambda x: " ".join([item for item in x if item in top_100_list_train]))
test['2tweet'] = test['tweet'].apply(lambda x: " ".join([item for item in x if item in top_100_list_test]))
dev['2tweet'] = dev['tweet'].apply(lambda x: " ".join([item for item in x if item in top_100_list_dev]))

In [14]:
len(train)

9447

In [15]:
train = train[train['2tweet']!=""]
del train['2tweet']
train.reset_index(inplace=True)
train.to_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/filtered_train.csv')

test = test[test['2tweet']!=""]
del test['2tweet']
test.reset_index(inplace=True)
test.to_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/filtered_test.csv')

dev = dev[dev['2tweet']!=""]
del dev['2tweet']
dev.reset_index(inplace=True)
dev.to_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/filtered_dev.csv')